In [33]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [34]:
df = pd.read_csv('./../Datasets/diabetes-data.csv', header=None)
df.dropna()
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [35]:
data_X = df.values[:, :-1]
data_Y = df.values[:, -1]
print(type(data_X), data_Y.shape)
df.corr()

<class 'numpy.ndarray'> (768,)


,0,1,2,3,4,5,6,7,8
0,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
1,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
2,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
3,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
4,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
5,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
6,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
7,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
8,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [36]:
train_X, test_X, train_Y, test_Y = train_test_split(data_X, data_Y, test_size=0.33, random_state=42)
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(514, 8) (514,) (254, 8) (254,)


In [45]:
def seperateByClass(dataset_X, dataset_Y):
    seperated = {}
    for i in range(dataset_X.shape[0]):
        if dataset_Y[i] not in seperated:
            seperated[dataset_Y[i]] = []
            seperated[dataset_Y[i]].append(dataset_X[i])
        else:
            seperated[dataset_Y[i]].append(dataset_X[i])
    return seperated, list(seperated.keys())
seperated, keys_seperated = seperateByClass(train_X, train_Y)

In [46]:
def mean(data):
    return sum(data)/float(len(data))
def stddev(data):
    av = mean(data)
    var = sum([pow(x-av, 2) for x in data])/float(len(data)-1)
    return math.sqrt(var)

In [47]:
def summarizeByClass(dataset):
    summary = {
        keys_seperated[0]:[],
        keys_seperated[1]:[]
    }
    for i in keys_seperated:
        for j in range(len(dataset[i][0])):
            arr_fed = [dataset[i][k][j] for k in range(len(dataset[i]))]
            av = mean(arr_fed)
            std = stddev(arr_fed)
            summary[keys_seperated[int(i)]].append([av, std])
    print(summary)
    return summary
summary = summarizeByClass(seperated)
        

{0.0: [[3.3042168674698793, 3.0016481355857976], [109.73493975903614, 25.23572700595366], [68.38253012048193, 17.7347371658133], [19.822289156626507, 14.602954121141789], [67.21385542168674, 94.6610077780182], [30.138554216867476, 7.237387509579614], [0.43497289156626523, 0.3109987748104605], [30.74397590361446, 11.345073321761951]], 1.0: [[4.653846153846154, 3.5749220968155444], [142.3846153846154, 33.32736311458942], [72.9065934065934, 18.18587456337677], [21.807692307692307, 16.986226014131134], [101.17032967032966, 144.10918435464254], [35.31043956043956, 7.257516646347943], [0.5426208791208794, 0.3835148497824337], [37.65384615384615, 11.119936878197453]]}


In [48]:
def pdf(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [62]:
def get_probability(inputVector):
    probablities = {}
    for j in keys_seperated:
        total_pdf = 1
        for i in range(len(summary[j])):
            m = summary[j][i][0]
            s = summary[j][i][1]
            feat_pdf = pdf(inputVector[i], m, s)
            total_pdf = total_pdf * feat_pdf
        probablities[j] = total_pdf
    return probablities
get_probability([8, 183, 64, 0, 0, 23.3, 0.672, 32])

{0.0: 8.396327524482848e-15, 1.0: 7.703743448292181e-14}

In [66]:
def get_probability_dataset(data):
    data_probabilities = []
    for i in range(data.shape[0]):
        prob = get_probability(data[i])
        if prob[keys_seperated[0]] > prob[keys_seperated[1]]:
            data_probabilities.append(keys_seperated[0])
        else:
            data_probabilities.append(keys_seperated[1])
    return data_probabilities
    
pred_Y = get_probability_dataset(test_X)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]


In [77]:
def get_accuracy(true_Y, pred_Y):
    count = 0
    for i in range(true_Y.shape[0]):
        if true_Y[i] == pred_Y[i]:
            count+=1
    acc = (float)(count/true_Y.shape[0])
    print("Accuracy : {}".format(acc))

In [78]:
get_accuracy(test_Y, pred_Y)

Accuracy : 0.7086614173228346
